In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import streamlit as st
import numpy as np
from glob import glob
import data_concat
from graph_data import show_login_form,transform_list, grf_plotly, one_angle_plotly,x_factor_plotly
from graph_data import kinematic_sequence_plotly, segment_power_plotly, linear_momentum_plotly,angular_momentum_plotly


KINEMATIC_PATH = 'data/**/kine/*csv'
FORCE_PATH = 'data/**/force/*csv'

KINEMATIC_DIR = [i.replace('\\','/') for i in glob(KINEMATIC_PATH)]
FORCE_DIR = [i.replace('\\','/') for i in glob(FORCE_PATH)]
csv_file = 'feedback.csv'
feedback_df = pd.read_csv(csv_file)
kdf = pd.DataFrame()
fdf = pd.DataFrame()

kdf = pd.DataFrame()
fdf = pd.DataFrame()
energy_cols = [
            "REAR_SHANK_NET_SP",    
            "LEAD_SHANK_NET_SP",
            "LEAD_THIGH_NET_SP",
            "REAR_THIGH_NET_SP",
            "PELVIS_NET_SP",
            "TORSO_NET_SP",
            "LEAD_ARM_NET_SP",
            "REAR_ARM_NET_SP",
            "LEAD_FOREARM_NET_SP",
            "REAR_FOREARM_NET_SP",
            ]
for kine_dir, force_dir in zip(KINEMATIC_DIR, FORCE_DIR):
    kine = pd.read_csv(kine_dir)
    force = pd.read_csv(force_dir)
    
    _, kday, _, kfname = kine_dir.split('/')
    _, fday, _, ffname = force_dir.split('/')
    kfname = kfname.replace('.csv','')
    kplayer_name, ktrial, _, mass, _, _, kball, _,kpit_type = kfname.split('_')
    
    ffname = ffname.replace('.csv','')
    fplayer_name, ftrial, _, mass, _, _, fball, _,fpit_type = ffname.split('_')
    
    lhjc = kine[['LEAD_HIP_JC_X','LEAD_HIP_JC_Y','LEAD_HIP_JC_Z']].values
    rhjc = kine[['REAR_HIP_JC_X','REAR_HIP_JC_Y','REAR_HIP_JC_Z']].values
    rsjc = kine[['REAR_SHOULDER_JC_X','REAR_SHOULDER_JC_Y','REAR_SHOULDER_JC_Z']].values
    lsjc = kine[['LEAD_SHOULDER_JC_X','LEAD_SHOULDER_JC_Y','LEAD_SHOULDER_JC_Z']].values
    pelvis_jc = pd.DataFrame((lhjc + rhjc)/2, columns = ['PELVIS_JC_X','PELVIS_JC_Y','PELVIS_JC_Z'])
    torso_jc = pd.DataFrame((rsjc + lsjc)/2, columns = ['TORSO_JC_X','TORSO_JC_Y','TORSO_JC_Z'])
    torso_pelvis_distance = pelvis_jc['PELVIS_JC_Y'] - torso_jc['TORSO_JC_Y']
    
    kine[energy_cols] = kine[energy_cols]/float(mass)
    kine['ANKLE_HAND_AP_DISTANCE'] = round(100 * (kine['LEAD_ANKLE_JC_Y'] - kine['LEAD_WRIST_JC_Y']),1)
    kine['PELVIS_TORSO_AP_DISTANCE'] = round(100 * torso_pelvis_distance,1)
    
    kine['player'] = kplayer_name
    kine['day'] = kday
    kine['trial'] = ktrial
    kine['ball_speed'] = kball
    kine['pit_type'] = kpit_type
    
    force['player'] = fplayer_name
    force['day'] = fday
    force['trial'] = ftrial
    force['ball_speed'] = fball
    force['pit_type'] = fpit_type
    
    kdf = pd.concat([kdf, kine])
    fdf = pd.concat([fdf, force])
    
# 스트림릿 사이드바 설정
selected_name = 'songseokhyun'; selected_date = '20240302'; selected_trial = '0014'

kine_filtered = kdf[(kdf['player'] == selected_name) & 
                    (kdf['day'] == selected_date) &
                    (kdf['trial'] == selected_trial)]

force_filtered = fdf[(fdf['player'] == selected_name) & 
                    (fdf['day'] == selected_date) &
                    (fdf['trial'] == selected_trial)]

kine_filtered.reset_index(inplace=True, drop=True)
force_filtered.reset_index(inplace=True, drop=True)

k_sr = 180
k_kh_time  = kine_filtered['kh_time'][0]
k_kh_time1 = kine_filtered['kh_time'][0] - k_kh_time
k_fc_time  = kine_filtered['fc_time'][0] - k_kh_time
k_mer_time = kine_filtered['mer_time'][0] - k_kh_time
k_br_time  = kine_filtered['br_time'][0] - k_kh_time
stride_length = round(kine_filtered['stride_length'][0])
ball_speed = round(float(kine_filtered['ball_speed'][0]) * 1.6)

f_sr = 1080
f_kh_time  = force_filtered['kh_time'][0] 
f_kh_time1 = force_filtered['kh_time'][0]  - f_kh_time
f_fc_time  = force_filtered['fc_time'][0]  - f_kh_time
f_mer_time = force_filtered['mer_time'][0] - f_kh_time
f_br_time  = force_filtered['br_time'][0]  - f_kh_time

k_df = kine_filtered.iloc[k_kh_time:int(k_br_time + k_kh_time + (k_sr * 0.2)),:].reset_index(drop=True)
f_df = force_filtered.iloc[f_kh_time:int(f_br_time + f_kh_time + (f_sr * 0.2)),:].reset_index(drop=True)

k_df.drop(['kh_time','fc_time','mer_time','br_time','mir_time'], axis=1, inplace=True)
f_df.drop(['kh_time','fc_time','mer_time','br_time','mir_time'], axis=1, inplace=True)

k_time = k_df['TIME']
f_time = f_df['TIME']

# ===================================================================================
# ============================= Using Data ==========================================
ap_cols = {
    'REAR_FORCE_Y' : ['Trail Leg' , 'yellow'],
    'LEAD_FORCE_Y' : ['Stride Leg', 'red'],
}
vt_cols = {
    'REAR_FORCE_Z' : ['Trail Leg' , 'yellow'],
    'LEAD_FORCE_Z' : ['Stride Leg', 'red']
}
result_cols = {
    'REAR_RESULT' : ['Trail Leg' , 'yellow'],
    'LEAD_RESULT' : ['Stride Leg', 'red']
}
momentum_cols = {
    'REAR_FREEMOMENT_Z' : ['Trail Leg' , 'yellow'],
    'LEAD_FREEMOMENT_Z' : ['Stride Leg', 'red']
}
ks_cols = {
    'PELVIS_SEG_ANGULAR_VELOCITY_Z'        : ['PELVIS'   , 'red'],
    'TORSO_SEG_ANGULAR_VELOCITY_Z'         : ['TORSO'    , 'green'],
    'LEAD_ELBOW_ANGULAR_VELOCITY_X'    : ['ELBOW'    , 'blue'],
    'LEAD_SHOULDER_ANGULAR_VELOCITY_Z' : ['SHOULDER' , 'yellow'],
}
ang_cols = {
    'TORSO_PELVIS_ANGLE_Z'            : 'HIP-SHOULDER SEPARATION',
    'LEAD_ELBOW_ANGLE_X'              : 'ELBOW FLEXION',
    'LEAD_SHOULDER_ANGLE_Z'           : 'SHOULDER EXTERNAL ROTATION',          
    'LEAD_SHOULDER_ANGLE_X'           : 'SHOULDER HORIZONTAL ABDUCTION',
    'LEAD_KNEE_ANGLE_X'               : 'LEAD KNEE FLEXION',
    'LEAD_KNEE_ANGULAR_VELOCITY_X'    : 'LEAD KNEE EXTENSION ANGULAR VELOCITY',
    'LEAD_SHOULDER_ANGLE_Y'           : 'SHOULDER ABDUCTION', 
    'TORSO_ANGLE_X'                   : 'TRUNK FORWARD TILT',
    'TORSO_ANGLE_Y'                   : 'TRUNK LATERAL TILT',
    'HAND_ELBOW_HEIGHT'               : 'HAND ELBOW HEIGHT',
    'TORSO_ANGLE_Z'                   : 'TRUNK ROTATION',
    'PELVIS_ANGLE_Z'                  : 'PELVIS ROTATION',
    'PELVIS_TORSO_AP_DISTANCE'        : 'PELVIS-HEAD DISTANCE',
    'ANKLE_HAND_AP_DISTANCE'          : 'ANKLE-HAND DISTANCE',
}
xfactor_cols = {
    'TORSO_PELVIS_ANGLE_Z'            : ['X-Factor', 'yellow'],
    'PELVIS_ANGLE_Z'                  : ['PELVIS', 'red'],
    'TORSO_ANGLE_Z'                   : ['TORSO', 'green'],
}
sp_cols = {
    "PELVIS_NET_SP" : ["PELVIS","red"],
    "TORSO_NET_SP" : ["TORSO","green"],
    "LEAD_ARM_NET_SP" : ["UPPER ARM","blue"],
    "LEAD_FOREARM_NET_SP" : ["FOREARM","yellow"],
}

lm_cols = {
    "PELVIS_LINEAR_MOMENTUM": ["PELVIS","red"],
    "TORSO_LINEAR_MOMENTUM": ["TORSO","green"],
    "LEAD_UPA_LINEAR_MOMENTUM": ["UPPER ARM","blue"],
    "LEAD_FA_LINEAR_MOMENTUM": ["FOREARM","yellow"],
}

am_cols = {
    "PELVIS_ANGULAR_MOMENTUM": ["PELVIS","red"],
    "TORSO_ANGULAR_MOMENTUM": ["TORSO","green"],
    "LEAD_UPA_ANGULAR_MOMENTUM": ["UPPER ARM","blue"],
    "LEAD_FA_ANGULAR_MOMENTUM": ["FOREARM","yellow"],
}

energy_cols = {
    # "LEAD_SHANK_LINEAR_MOMENTUM": ["SHANK LINEAR MOMENTUM","shank_energy"],
    # "LEAD_THIGH_LINEAR_MOMENTUM": ["THIGH LINEAR MOMENTUM","thigh_energy"],
    "PELVIS_LINEAR_MOMENTUM": ["PELVIS LINEAR MOMENTUM","pelvis_energy"],
    "TORSO_LINEAR_MOMENTUM": ["TORSO LINEAR MOMENTUM","torso_energy"],
    "LEAD_UPA_LINEAR_MOMENTUM": ["ARM LINEAR MOMENTUM","arm_energy"],
    "LEAD_FA_LINEAR_MOMENTUM": ["FOREARM LINEAR MOMENTUM","forearm_energy"],
    # "LEAD_SHANK_ANGULAR_MOMENTUM": ["SHANK ANGULAR MOMENTUM","shank_energy"],
    # "LEAD_THIGH_ANGULAR_MOMENTUM": ["THIGH ANGULAR MOMENTUM","thigh_energy"],
    "PELVIS_ANGULAR_MOMENTUM_Z": ["PELVIS ANGULAR MOMENTUM","pelvis_energy"],
    "TORSO_ANGULAR_MOMENTUM_Z": ["TORSO ANGULAR MOMENTUM","torso_energy"],
    "LEAD_UPA_ANGULAR_MOMENTUM": ["ARM ANGULAR MOMENTUM","arm_energy"],
    "LEAD_FA_ANGULAR_MOMENTUM": ["FOREARM ANGULAR MOMENTUM","forearm_energy"],
    # "LEAD_SHANK_NET_SP" : ["SHANK POWER","shank_energy"],
    # "LEAD_THIGH_NET_SP" : ["THIGH POWER","thigh_energy"],
    "PELVIS_NET_SP" : ["PELVIS POWER","pelvis_energy"],
    "TORSO_NET_SP" : ["TORSO POWER","torso_energy"],
    "LEAD_ARM_NET_SP" : ["ARM POWER","arm_energy"],
    "LEAD_FOREARM_NET_SP" : ["FOREARM POWER","forearm_energy"],
}

In [3]:
import numpy as np
np.__version__

'2.1.3'

In [5]:
force_ap_fig, force_ap_values = grf_plotly(f_df, ap_cols, f_time, f_kh_time1, f_fc_time, f_mer_time, f_br_time, axis='ap')
force_vt_fig, force_vt_values = grf_plotly(f_df, vt_cols, f_time, f_kh_time1, f_fc_time, f_mer_time, f_br_time, axis='vt')
force_result_fig, force_result_values = grf_plotly(f_df, result_cols, f_time, f_kh_time1, f_fc_time, f_mer_time, f_br_time, axis='result')
force_momentum_fig, force_momentum_values = grf_plotly(f_df, momentum_cols, f_time, f_kh_time1, f_fc_time, f_mer_time, f_br_time, axis='freemoment')
kine_values, kine_fig = one_angle_plotly(k_df, ang_cols, k_time, k_kh_time1, k_fc_time, k_mer_time, k_br_time)
kinematic_values, kinematic_fig = kinematic_sequence_plotly(k_df, ks_cols, k_time, k_kh_time1, k_fc_time, k_mer_time, k_br_time)
segment_power_values, segment_power_fig = segment_power_plotly(k_df, sp_cols, k_time, k_kh_time1, k_fc_time, k_mer_time, k_br_time)
linear_momentum_values, linear_momentum_fig = linear_momentum_plotly(k_df, lm_cols, k_time, k_kh_time1, k_fc_time, k_mer_time, k_br_time)
angular_momentum_values, angular_momentum_fig = angular_momentum_plotly(k_df, am_cols, k_time, k_kh_time1, k_fc_time, k_mer_time, k_br_time)
xfactor_values, xfactor_fig = angular_momentum_plotly(k_df, xfactor_cols, k_time, k_kh_time1, k_fc_time, k_mer_time, k_br_time)
# energy_values, energy_fig = energy_plotly(k_df, energy_cols, k_time, k_kh_time1, k_fc_time, k_mer_time, k_br_time)



In [6]:
angular_momentum_values

{'max': {'PELVIS_ANGULAR_MOMENTUM': 0.34,
  'TORSO_ANGULAR_MOMENTUM': 10.08,
  'LEAD_UPA_ANGULAR_MOMENTUM': 0.61,
  'LEAD_FA_ANGULAR_MOMENTUM': 0.45},
 'max_time': {'PELVIS_ANGULAR_MOMENTUM': 164,
  'TORSO_ANGULAR_MOMENTUM': 158,
  'LEAD_UPA_ANGULAR_MOMENTUM': 181,
  'LEAD_FA_ANGULAR_MOMENTUM': 172},
 'fc_time': {'PELVIS_ANGULAR_MOMENTUM': 0.22,
  'TORSO_ANGULAR_MOMENTUM': 5.07,
  'LEAD_UPA_ANGULAR_MOMENTUM': 0.16,
  'LEAD_FA_ANGULAR_MOMENTUM': 0.12},
 'mer_time': {'PELVIS_ANGULAR_MOMENTUM': 0.33,
  'TORSO_ANGULAR_MOMENTUM': 8.66,
  'LEAD_UPA_ANGULAR_MOMENTUM': 0.37,
  'LEAD_FA_ANGULAR_MOMENTUM': 0.3},
 'br_time': {'PELVIS_ANGULAR_MOMENTUM': 0.12,
  'TORSO_ANGULAR_MOMENTUM': 5.44,
  'LEAD_UPA_ANGULAR_MOMENTUM': 0.23,
  'LEAD_FA_ANGULAR_MOMENTUM': 0.42},
 'min': {'PELVIS_ANGULAR_MOMENTUM': 0.02,
  'TORSO_ANGULAR_MOMENTUM': 0.34,
  'LEAD_UPA_ANGULAR_MOMENTUM': 0.01,
  'LEAD_FA_ANGULAR_MOMENTUM': 0.0},
 'min_time': {'PELVIS_ANGULAR_MOMENTUM': 4,
  'TORSO_ANGULAR_MOMENTUM': 101,
  'LEAD_UP

In [74]:
ENERGY = []
for c in kine_filtered.columns:
    if 'LINEAR_MOMENTUM' in c:
        print(c)


REAR_FA_LINEAR_MOMENTUM_X
REAR_FA_LINEAR_MOMENTUM_Y
REAR_FA_LINEAR_MOMENTUM_Z
LEAD_FOOT_LINEAR_MOMENTUM_X
LEAD_FOOT_LINEAR_MOMENTUM_Y
LEAD_FOOT_LINEAR_MOMENTUM_Z
REAR_HAND_LINEAR_MOMENTUM_X
REAR_HAND_LINEAR_MOMENTUM_Y
REAR_HAND_LINEAR_MOMENTUM_Z
LEAD_SHANK_LINEAR_MOMENTUM_X
LEAD_SHANK_LINEAR_MOMENTUM_Y
LEAD_SHANK_LINEAR_MOMENTUM_Z
LEAD_THIGH_LINEAR_MOMENTUM_X
LEAD_THIGH_LINEAR_MOMENTUM_Y
LEAD_THIGH_LINEAR_MOMENTUM_Z
REAR_UPA_LINEAR_MOMENTUM_X
REAR_UPA_LINEAR_MOMENTUM_Y
REAR_UPA_LINEAR_MOMENTUM_Z
PELVIS_LINEAR_MOMENTUM_X
PELVIS_LINEAR_MOMENTUM_Y
PELVIS_LINEAR_MOMENTUM_Z
LEAD_FA_LINEAR_MOMENTUM_X
LEAD_FA_LINEAR_MOMENTUM_Y
LEAD_FA_LINEAR_MOMENTUM_Z
REAR_FOOT_LINEAR_MOMENTUM_X
REAR_FOOT_LINEAR_MOMENTUM_Y
REAR_FOOT_LINEAR_MOMENTUM_Z
LEAD_HAND_LINEAR_MOMENTUM_X
LEAD_HAND_LINEAR_MOMENTUM_Y
LEAD_HAND_LINEAR_MOMENTUM_Z
REAR_SHANK_LINEAR_MOMENTUM_X
REAR_SHANK_LINEAR_MOMENTUM_Y
REAR_SHANK_LINEAR_MOMENTUM_Z
REAR_THIGH_LINEAR_MOMENTUM_X
REAR_THIGH_LINEAR_MOMENTUM_Y
REAR_THIGH_LINEAR_MOMENTUM_Z
LEAD

In [19]:
lin_mmt_cols = {
     "LEAD_SHANK_LINEAR_MOMENTUM": ["SHANK", "#A4C2F4"],
     "LEAD_THIGH_LINEAR_MOMENTUM": ["THIGH", "#A4F4A4"],
     "PELVIS_LINEAR_MOMENTUM": ["PELVIS", "#FFF58F"],
     "TORSO_LINEAR_MOMENTUM": ["TORSO", "#FFB6C1"],
     "LEAD_UPA_LINEAR_MOMENTUM": ["ARM", "#C8A2C8"],
     "LEAD_FA_LINEAR_MOMENTUM": ["FOREARM", "#FFD1A4"],
}

ang_mmt_cols = {
     "LEAD_SHANK_ANGULAR_MOMENTUM": ["SHANK", "#A4C2F4"],
     "LEAD_THIGH_ANGULAR_MOMENTUM": ["THIGH", "#A4F4A4"],
     "PELVIS_ANGULAR_MOMENTUM": ["PELVIS", "#FFF58F"],
     "TORSO_ANGULAR_MOMENTUM": ["TORSO", "#FFB6C1"],
     "LEAD_UPA_ANGULAR_MOMENTUM": ["ARM", "#C8A2C8"],
     "LEAD_FA_ANGULAR_MOMENTUM": ["FOREARM", "#FFD1A4"],
}
energy_cols = {
    "LEAD_SHANK_NET_SP" : ["SHANK", "#A4C2F4"],
    "LEAD_THIGH_NET_SP" : ["THIGH", "#A4F4A4"],
    "PELVIS_NET_SP" : ["PELVIS", "#FFF58F"],
    "TORSO_NET_SP" : ["TORSO", "#FFB6C1"],
    "LEAD_ARM_NET_SP" : ["ARM", "#C8A2C8"],
    "LEAD_FOREARM_NET_SP" : ["FOREARM", "#FFD1A4"],
}

energy_cols = {
     "LEAD_SHANK_LINEAR_MOMENTUM": "SHANK LINEAR MOMENTUM",
     "LEAD_THIGH_LINEAR_MOMENTUM": "THIGH LINEAR MOMENTUM",
     "PELVIS_LINEAR_MOMENTUM": "PELVIS LINEAR MOMENTUM",
     "TORSO_LINEAR_MOMENTUM": "TORSO LINEAR MOMENTUM",
     "LEAD_UPA_LINEAR_MOMENTUM": "ARM LINEAR MOMENTUM",
     "LEAD_FA_LINEAR_MOMENTUM": "FOREARM LINEAR MOMENTUM",
     "LEAD_SHANK_ANGULAR_MOMENTUM": "SHANK ANGULAR MOMENTUM",
     "LEAD_THIGH_ANGULAR_MOMENTUM": "THIGH ANGULAR MOMENTUM",
     "PELVIS_ANGULAR_MOMENTUM": "PELVIS ANGULAR MOMENTUM",
     "TORSO_ANGULAR_MOMENTUM": "TORSO ANGULAR MOMENTUM",
     "LEAD_UPA_ANGULAR_MOMENTUM": "ARM ANGULAR MOMENTUM",
     "LEAD_FA_ANGULAR_MOMENTUM": "FOREARM ANGULAR MOMENTUM",
     "LEAD_SHANK_NET_SP" : "SHANK POWER",
     "LEAD_THIGH_NET_SP" : "THIGH POWER",
     "PELVIS_NET_SP" : "PELVIS POWER",
     "TORSO_NET_SP" : "TORSO POWER",
     "LEAD_ARM_NET_SP" : "ARM POWER",
     "LEAD_FOREARM_NET_SP" : "FOREARM POWER",
}

In [15]:
import plotly.graph_objs as go
def energy_plotly(data, cols, time, k_kh_time, k_fc_time, k_mer_time, k_br_time):
    ang = {
        'max'       : {},
        'max_time'  : {},
        'min'       : {},
        'min_time'  : {},
        'fc_time'   : {},
        'mer_time'  : {},
        'br_time'   : {},
    }
    
    figures = {}
    
    for col in cols:
        df = data[col]
        if 'LINEAR_MOMENTUM' in col:
            y_label = 'Momentum [kg*m²/s]'
        elif 'ANGULAR_MOMENTUM' in col:
            y_label = 'Momentum [kg*m²/(s*rad)]'
        elif 'NET' in col:
            y_label = 'Power [W/kg]'
        # Create the trace for the main data line
        trace = go.Scatter(x=time, y=df, mode='lines', name=cols[col], line=dict(color='firebrick', width=4))
        traces = [trace]
        
        ang['fc_time'][col]   = round(df[k_fc_time], 2)
        ang['mer_time'][col]  = round(df[k_mer_time], 2)
        ang['br_time'][col]   = round(df[k_br_time], 2)
        ang['max'][col]       = round(df.max(), 2)
        ang['max_time'][col] = np.where(df == df.min())[0][0]
        ang['min'][col]       = round(df.max(), 2)
        ang['min_time'][col] = np.where(df == df.min())[0][0]
        
        event_times = [k_kh_time, k_fc_time, k_mer_time, k_br_time]
        event_names = ['KH', 'FC', 'MER', 'BR']
        shapes = [
            {
                'type': 'line',
                'xref': 'x',
                'yref': 'paper',
                'x0': time[event_time],
                'y0': 0,
                'x1': time[event_time],
                'y1': 1,
                'opacity' : 0.5,
                'line': {
                    'color': 'cyan',
                    'width': 4,
                    'dash': 'dash',
                }
            } for event_time in event_times
        ]
        annotations = [
            {
                'x': time[event_time + 2],
                'y': 1,
                'xref': 'x',
                'yref': 'paper',
                'text': label,
                'showarrow': False,
                'font': {
                    'color': 'cyan',
                    'size' : 16
                },
                'textangle': -90
            } for event_time, label in zip(event_times, event_names)
        ]
        
        # Define the layout
        layout = go.Layout(
            title=f'{cols[col]}',
            xaxis=dict(title='Time [s]',
                       showgrid=False),
            yaxis=dict(title=y_label,
                       autorange = True,
                       rangemode='tozero',
                        showgrid=True,         # This will show the horizontal gridlines
                        gridcolor='lightgrey',
                        gridwidth=1,
                        zeroline=False,
                        ),                        
            showlegend=False,
            shapes =shapes,
            margin=dict(l=40, r=40, t=40, b=40),
            height=600,
            plot_bgcolor='rgb(43,48,61)',
            annotations=annotations
        )
        
        # Create the figure and add the traces to it
        fig = go.Figure(data=traces, layout=layout)
        
        # Store the figure in the dictionary
        figures[col] = fig
        
    return ang, figures


In [28]:
def save_feedback(df, csv_file, name, date, trial, feedback):
    new_feedback = pd.DataFrame({
        'name' : [name],
        'date' : [date],
        'trial' : [trial],
        'feedback' : [feedback]})
    
    df_updated = pd.concat([df,new_feedback], ignore_index=True)
    df_updated.to_csv(csv_file, index=False)
    
    return df_updated

# save_feedback(feedback_df, csv_file, selected_name, selected_date, selected_trial, feedback_input)
feedback_df = pd.read_csv(csv_file)
filtered_feedback = [i for i in feedback_df[(feedback_df['name'] == selected_name) & 
                                        (feedback_df['date'] == int(selected_date)) & 
                                        (feedback_df['trial'] == int(selected_trial))]['feedback'].values]

In [34]:
[i for i in filtered_feedback.values]

["['피드백 진행 사항 이렇게 진행되는게 맞는건지?\\n                  근데  이게 맞아?']",
 '피드백 진행 사항 이렇게 진행되는게 맞는건지?  근데 이게 맞아?\n뭔가 이상한데?']

In [18]:
feedback_df[(feedback_df['name'] == 'Kangmingu') & 
                                        (feedback_df['date'] == int('20240324')) & 
                                        (feedback_df['trial'] == 1)]['feedback']

0    피드백 남기기
Name: feedback, dtype: object

In [17]:
kinematic_values, kinematic_fig = energy_plotly(kine_filtered, energy_cols, k_time, k_kh_time1, k_fc_time, k_mer_time, k_br_time)


In [18]:
for i in kinematic_fig:
    display(kinematic_fig[i])